In [44]:
import networkx as nx
import geopandas as gpd
import shapely

import os

DATA_DIR = '/mnt/c/Users/natha/Documents/ArcGIS/Projects/ClimateRailnet'
os.chdir(DATA_DIR)

In [10]:
railnet = nx.read_gml('Data/faf_railnet.gml')

In [30]:
faf_network_nodes_shapefile_name = f'{os.getcwd()}/Data/faf_network_nodes.shp'
faf_network_nodes_gdf = gpd.GeoDataFrame.from_records([railnet.nodes[i] for i in railnet.nodes])
faf_network_nodes_gdf = faf_network_nodes_gdf.set_geometry(gpd.points_from_xy(faf_network_nodes_gdf.lon, faf_network_nodes_gdf.lat))
faf_network_nodes_gdf = faf_network_nodes_gdf.set_index('faf_id')
faf_network_nodes_gdf = faf_network_nodes_gdf.drop(columns=['lon', 'lat'], axis=1)
faf_network_nodes_gdf = faf_network_nodes_gdf.set_crs(epsg=4269)
faf_network_nodes_gdf.to_file(faf_network_nodes_shapefile_name, )
faf_network_nodes_gdf

,name,geometry
faf_id,,
361,"Albany-Schenectady, NY CFS Area",POINT (-73.85539 43.10614)
131,"Atlanta-Athens-Clarke County-Sandy Springs, GA...",POINT (-84.14672 33.07446)
481,"Austin-Round Rock, TX CFS Area",POINT (-97.62814 29.8324)
241,"Baltimore-Columbia-Towson, MD CFS Area",POINT (-76.61048 39.30003)
221,"Baton Rouge, LA CFS Area",POINT (-91.04343 30.83978)
...,...,...
512,"Virginia Beach-Norfolk, VA-NC CFS Area (VA Part)",POINT (-76.30353 37.12836)
111,"Washington-Arlington-Alexandria, DC-VA-MD-WV ...",POINT (-77.01723 38.9041)
242,"Washington-Arlington-Alexandria, DC-VA-MD-WV ...",POINT (-76.52949 38.52261)


In [55]:
faf_network_nodes_gdf.loc[361]['name']

'Albany-Schenectady, NY  CFS Area'

In [54]:
faf_network_edges_shapefile_name = f'{os.getcwd()}/Data/faf_network_edges.shp'

edge_list = []
for fro, to in railnet.edges:
  fro_node = faf_network_nodes_gdf.loc[int(fro)]
  to_node = faf_network_nodes_gdf.loc[int(to)]
  
  edge_list.append({
    'to': to,
    'fro': fro,
    'weight': railnet.edges[(fro,to)]['weight'],
    'geometry': shapely.LineString((
      (fro_node.geometry.x,fro_node.geometry.y),
      (to_node.geometry.x,to_node.geometry.y)
    ))
  })

faf_network_edges_gdf = gpd.GeoDataFrame(edge_list)
faf_network_edges_gdf = faf_network_edges_gdf.set_crs(epsg=4269)
faf_network_edges_gdf.to_file(faf_network_edges_shapefile_name, )
faf_network_edges_gdf

,to,fro,weight,geometry
0,500,361,3,"LINESTRING (-73.85539 43.10614, -72.9094 44.85..."
1,369,361,2,"LINESTRING (-73.85539 43.10614, -77.38553 42.2..."
2,363,361,1,"LINESTRING (-73.85539 43.10614, -74.26546 41.9..."
3,259,361,1,"LINESTRING (-73.85539 43.10614, -72.66369 42.3..."
4,139,131,10,"LINESTRING (-84.14672 33.07446, -82.47873 33.4..."
...,...,...,...,...
250,513,511,1,"LINESTRING (-77.49132 37.76021, -77.45488 38.4..."
251,64,62,1,"LINESTRING (-121.34044 38.45002, -122.92611 38..."
252,292,172,3,"LINESTRING (-89.43659 38.88592, -91.07341 38.4..."
253,242,111,1,"LINESTRING (-77.01723 38.9041, -76.52949 38.52..."
